In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc
from xgboost import XGBClassifier

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisosONEHOT.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantesONEHOT.csv")
avisos['nombre_area']=avisos['nombre_area'].astype('category')

avisos_no_one_hot= pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
avisos_no_one_hot=avisos_no_one_hot[['idaviso','tipo_de_trabajo','nivel_laboral']];
avisos_no_one_hot['tipo_de_trabajo']=avisos_no_one_hot['tipo_de_trabajo'].astype('category')
avisos_no_one_hot['nivel_laboral']=avisos_no_one_hot['nivel_laboral'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas2=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2=vistas2.rename(columns={'idAviso': 'idaviso'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [7]:
vistas2=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [8]:
vistas2.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [9]:
vistas2['cantidad']=vistas2['cantidad_x']/vistas2['cantidad_y']
vistas2.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas=vistas2
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [10]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/7.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/8.csv").sample(len(postulaciones2)//2).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2

9906801


,idaviso,idpostulante,sepostulo
0,1112389495,5NbOjG,0
1,1112024109,1QP841b,0
2,1112490884,96XQ6N9,0
3,1111474259,qe2QJj1,0
4,1112211292,KBzbBJm,0
5,1112328485,3NzZv26,0
6,1112474766,akDjWwE,0
7,1112312385,aZpBJW,0
8,1112285628,akDWj55,0
9,1112322843,GNd0KNN,0


In [11]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [12]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16502017


In [13]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,avisos_no_one_hot,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,tipo_de_trabajo_Temporario,tipo_de_trabajo_Voluntario,nivel_laboral_Gerencia / Alta Gerencia / Dirección,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,cantidad
13313219,1112284561,0zNKwVY,1,22.0,False,True,False,False,False,False,...,False,False,False,False,True,False,False,Full-time,Junior,NaN
15647069,1112056256,ZDPNmxz,0,27.0,True,False,False,False,False,False,...,False,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,NaN
2812823,1112298982,GNJYE2w,1,27.0,False,True,False,False,False,False,...,False,False,False,False,True,False,False,Part-time,Junior,NaN
1686352,1112278312,0zrmvdE,0,24.0,False,True,False,False,False,False,...,False,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,NaN
3703463,1112274890,EmEM1b,0,27.0,True,False,False,False,False,False,...,False,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,NaN
2951563,1112430106,VNrvRBl,0,24.0,True,False,False,False,False,False,...,False,False,False,False,True,False,False,Full-time,Junior,NaN
7111285,1112298389,JBx0Mr5,0,35.0,False,True,False,False,False,False,...,False,False,False,False,False,True,False,Part-time,Otro,NaN
889575,1112352852,eAv5O8,1,25.0,True,False,False,False,False,False,...,False,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,NaN
6176092,1112296264,6rP1Yl3,1,19.0,True,False,False,False,False,False,...,False,False,False,False,True,False,False,Full-time,Junior,NaN
5243223,1112371798,1QPZod3,1,18.0,False,True,False,False,False,False,...,False,False,False,False,False,False,True,Part-time,Senior / Semi-Senior,NaN


In [14]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,tipo_de_trabajo_Temporario,tipo_de_trabajo_Voluntario,nivel_laboral_Gerencia / Alta Gerencia / Dirección,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,cantidad
1076793,1112353428,1PLG8j,1,32.0,False,True,False,False,False,False,...,False,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000
10931393,1112355306,W9A98JR,1,21.0,False,True,False,False,False,False,...,False,False,False,False,True,False,False,Full-time,Junior,0.007407
7895759,1112418480,GNJEvrE,0,25.0,False,True,False,False,False,False,...,False,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000
7050537,1112237498,EzekGK4,1,22.0,True,False,False,False,False,False,...,False,False,False,False,False,False,True,Part-time,Senior / Semi-Senior,0.000000
99510,1112192709,PmVrRMa,0,30.0,False,True,False,False,False,False,...,False,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000
7780220,1111456309,mzblv4G,0,36.0,False,True,False,False,False,False,...,False,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000
2158546,1111753681,KBrYa5m,1,21.0,True,False,False,False,False,False,...,False,False,False,False,False,False,True,Full-time,Senior / Semi-Senior,0.007752
3417026,1112409725,wVkpP9V,1,29.0,True,False,False,False,False,False,...,False,False,False,False,True,False,False,Full-time,Junior,0.150685
4708898,1112363380,mzV4m1E,0,27.0,True,False,False,False,False,False,...,False,False,False,True,False,False,False,Full-time,Jefe / Supervisor / Responsable,0.000000
12113219,1112450798,Nz06JB4,0,42.0,False,True,False,False,False,False,...,False,False,False,True,False,False,False,Full-time,Jefe / Supervisor / Responsable,0.000000


In [15]:
postulaciones_por_area=postulaciones.copy()
postulaciones_por_area=postulaciones_por_area.groupby(by=['idpostulante','nombre_area'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_area=postulaciones_por_area.rename(columns={'sepostulo':'postulaciones_misma_area'})
postulaciones_por_area.head()

,idpostulante,nombre_area,postulaciones_misma_area
0,0z5Dmrd,Administración,1
1,0z5Dmrd,Almacén / Depósito / Expedición,1
2,0z5Dmrd,Atención al Cliente,1
3,0z5Dmrd,Auditoría,1
4,0z5Dmrd,Comercial,1


In [16]:
postulaciones_por_nivel=postulaciones.copy()
postulaciones_por_nivel=postulaciones_por_nivel.groupby(by=['idpostulante','nivel_laboral'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_nivel=postulaciones_por_nivel.rename(columns={'sepostulo':'postulaciones_mismo_nivel'})
postulaciones_por_nivel.head()

,idpostulante,nivel_laboral,postulaciones_mismo_nivel
0,0z5Dmrd,-,1
1,0z5Dmrd,Gerencia / Alta Gerencia / Dirección,1
2,0z5Dmrd,Junior,4
3,0z5Dmrd,Otro,1
4,0z5Dmrd,Senior / Semi-Senior,16


In [17]:
postulaciones_por_tipo=postulaciones.copy()
postulaciones_por_tipo=postulaciones_por_tipo.groupby(by=['idpostulante','tipo_de_trabajo'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_tipo=postulaciones_por_tipo.rename(columns={'sepostulo':'postulaciones_mismo_tipo'})
postulaciones_por_tipo.head()

,idpostulante,tipo_de_trabajo,postulaciones_mismo_tipo
0,0z5Dmrd,Full-time,21
1,0z5Dmrd,Part-time,1
2,0z5Dmrd,Teletrabajo,1
3,0z5JW1r,Full-time,30
4,0z5VvGv,Full-time,56


In [18]:
postulaciones=pd.merge(postulaciones,postulaciones_por_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['postulaciones_misma_area']=postulaciones['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_nivel']=postulaciones['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_tipo']=postulaciones['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,cantidad,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
14492543,1112493637,8MaqRoz,0,28.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000,1,81,57
8764859,1112432857,GNX9joE,1,38.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,0.095238,0,23,15
12112199,1112277793,82mXqR,0,37.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000,0,392,316
14322579,1112235784,QNa3GYW,0,41.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000,0,18,16
6702633,1111671997,QNrWL2z,0,24.0,True,False,False,False,False,False,...,False,True,False,False,Temporario,Junior,0.000000,0,0,4
2814299,1112322532,OqbkO0N,1,37.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000,3,21,15
13450552,1112245368,W9ONqX2,0,21.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000,0,15,13
9883689,1112263175,BmZ6jkR,0,22.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000,2,37,27
7173284,1112297717,0zPb8m1,1,24.0,True,False,False,False,False,False,...,False,False,True,False,Full-time,Otro,0.000000,1,32,1
7008999,1112320214,jk6jLvY,1,22.0,True,False,False,False,False,False,...,False,True,False,False,Full-time,Junior,0.040000,2,39,8


In [19]:
areas=postulaciones['nombre_area'].value_counts()
areas.values.mean()

85564.86170212766

In [22]:
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
vistas2=0

Train:  15281884 Test:  804310


In [35]:
c=0
gc.collect()

2559

In [36]:
areas

Ventas                                       2094769
Administración                               1250646
Comercial                                    1132045
Producción                                   1107931
Atención al Cliente                           652671
Recepcionista                                 424151
Contabilidad                                  420893
Call Center                                   405597
Programación                                  390351
Tecnologia / Sistemas                         329715
Almacén / Depósito / Expedición               263889
Telemarketing                                 263098
Mantenimiento                                 260730
Recursos Humanos                              256705
Tesorería                                     251378
Logística                                     242354
Gastronomia                                   242243
Mantenimiento y Limpieza                      233189
Soporte Técnico                               

In [ ]:
for i in range(0,len(areas)):
    if(areas.values[i])>100000:
        c=lgb.LGBMClassifier(learning_rate=0.01,objective='binary',num_leaves=6000,max_depth=18,n_estimators=2*int(round(areas.values[i]*0.0001)),colsample_bytree=0.8,n_jobs=-1,random_state=0,silent=False,subsample=0.8,sumsample_freq=0.5)
        dt=c.fit(x_train[x_train['nombre_area']==areas.index[i]],y_train[x_train['nombre_area']==areas.index[i]])
        joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/LGBMPorArea/LGBM "+areas.index[i].replace("/","-")+".pkl")
        score=c.score(x_test[x_test['nombre_area']==areas.index[i]],y_test[x_test['nombre_area']==areas.index[i]])*100
        print(areas.index[i]+": "+str(score))
        c=0
        gc.collect()

In [62]:
c=0
gc.collect()

317

In [54]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos_no_one_hot,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion=pd.merge(prediccion,postulaciones_por_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['postulaciones_misma_area']=prediccion['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion['postulaciones_mismo_nivel']=prediccion['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion['postulaciones_mismo_tipo']=prediccion['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion.sample(10)

100000
100000
100000
100000


,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Jefe / Supervisor / Responsable,nivel_laboral_Junior,nivel_laboral_Otro,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,cantidad,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
41053,45325,1112428153,LNwWZ45,40.0,False,True,False,False,False,False,...,False,True,False,False,Full-time,Junior,0.111111,14.0,36.0,10.0
1586,86711,1112466769,5pxZLw,28.0,False,True,False,False,False,False,...,False,True,False,False,Full-time,Junior,0.007519,0.0,78.0,13.0
48864,89952,1112467173,5mqMkeq,25.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,0.005391,16.0,109.0,83.0
18093,73526,1112463356,mzdlA4d,39.0,True,False,False,False,False,False,...,False,False,True,False,Part-time,Otro,0.028571,2.0,1.0,4.0
67129,39227,1112417321,1pGRqQ,32.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000,0.0,99.0,68.0
20294,81695,1112466106,2zPb6Jw,20.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000,7.0,295.0,203.0
24696,41225,1112420600,0zPVvVM,19.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000,11.0,106.0,83.0
96640,96289,1112493595,851v2j,36.0,False,True,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,0.000000,2.0,18.0,18.0
40410,48776,1112436970,ekOqJ6E,22.0,True,False,False,False,False,False,...,False,True,False,False,Part-time,Junior,0.068966,1.0,6.0,6.0
401,66973,1112460141,mzGAr1G,21.0,True,False,False,False,False,False,...,False,False,False,True,Full-time,Senior / Semi-Senior,0.133333,3.0,23.0,16.0


In [55]:
#Elijo una buena prediccion
buena = pd.read_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion35.csv")
prediccion=pd.merge(prediccion, buena,on='id',how='left')

In [56]:
for i in range(0,len(areas)):
    if(areas.values[i])>100000:
        c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/LGBMPorArea/LGBM "+areas.index[i].replace("/","-")+".pkl")
        nueva=prediccion[prediccion['nombre_area']==areas.index[i]]
        x=nueva[features]
        y=c.predict_proba(x)
        y2=[]
        for elem in y:
            y2.append(elem[1])
        nueva['sepostulo']=y2
        prediccion=pd.concat([prediccion, nueva],axis=0,ignore_index=True)
        prediccion.drop_duplicates('id',keep='last',inplace=True)
        c=0
        x=0
        y=0
        y2=0
        nueva=0
        gc.collect()
        print(areas.index[i])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


Ventas
Administración
Comercial
Producción
Atención al Cliente
Recepcionista
Contabilidad
Call Center
Programación
Tecnologia / Sistemas
Almacén / Depósito / Expedición
Telemarketing
Mantenimiento
Recursos Humanos
Tesorería
Logística
Gastronomia
Mantenimiento y Limpieza
Soporte Técnico
Oficios y Profesiones
Marketing
Administración de Personal
Compras
Créditos y Cobranzas
Finanzas
Legal
Transporte
Otros
Seguridad
Secretaria
Salud
Selección
Desarrollo de Negocios
Calidad
Impuestos


In [59]:
prediccion.sort_values('id',inplace=True)
prediccion.head()

,id,sepostulo
42715,0,0.179996
42716,1,0.326732
42717,2,0.189662
71648,3,0.364597
71649,4,0.360560


In [58]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

ValueError: labels ['tipo_de_trabajo' 'nivel_laboral' 'nombre_area'] not contained in axis

In [61]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion41.csv",index=False)

In [60]:
prediccion.sepostulo.mean()

0.4885296151602945